## Bibliotecas Utilizadas no Código

No código abaixo, são importadas várias bibliotecas para fornecer funcionalidades específicas, que vão desde a manipulação de dados e integração com o MetaTrader 5 até a criação de interfaces gráficas.

In [ ]:
# bibliotecas

import MetaTrader5 as mt5 # Permite a conexão com a plataforma MT5 para coletar dados e enviar ordens de compra e venda.
import pandas as pd # Essencial para manipulação de dados financeiros e organização das informações coletadas.
from datetime import datetime # Utilizado para gerenciar datas e horários dentro do processo de coleta.
import time # Fornece diversas funções para manipulação do tempo.
import os # Facilita a manipulação de diretórios e arquivos do sistema.

# Responsáveis por criar interfaces gráficas para a seleção e salvamento de arquivos.
import tkinter as tk 
from tkinter import filedialog

from IPython.display import clear_output # Limpa a saída da célula de código, ou seja, remove a última impressão ou resultado exibido no notebook.

import random # Gera números aleatórios, caso necessário para testes ou simulações.

### AQUI EXECUTAMOS O COMANDO QUE ABRE O METATRADER 5

In [2]:
# Função para conectar ao MetaTrader 5
def connect_mt5():
    if not mt5.initialize():
        print("Erro ao inicializar MetaTrader 5.")
        mt5.shutdown()

## Função para Enviar Ordens de Compra e Venda no MetaTrader 5

Este código define duas funções para enviar ordens de compra e venda limitadas na plataforma MetaTrader 5:

- **`send_buy_limit_order`**: Envia uma ordem de **compra** limitada para um ativo específico. A ordem é enviada com parâmetros como o símbolo do ativo, volume da ordem, preço desejado, stop loss, take profit e outras configurações relacionadas à execução da ordem.

- **`send_sell_limit_order`**: Envia uma ordem de **venda** limitada para um ativo, com a mesma estrutura de parâmetros da ordem de compra, incluindo o símbolo, volume, preço, stop loss e take profit.

Ambas as funções:
1. Estruturam os dados da ordem conforme a API do MetaTrader 5.
2. Enviam a ordem usando a função `order_send`.
3. Verificam o código de retorno (`retcode`) da operação para garantir que a ordem foi enviada corretamente. Caso contrário, um erroelo usuário.


In [ ]:
def send_buy_limit_order(symbol, volume, price, stop_loss, take_profit):
    # Estrutura da Ordem de Compra
    order = {
        "action": mt5.TRADE_ACTION_DEAL,  # Ordem imediata
        "symbol": symbol,  # Símbolo do ativo
        "volume": volume,  # Volume da ordem
        "type": mt5.ORDER_TYPE_BUY,  # Tipo de ordem
        "price": price,  # Preço
        "sl": stop_loss,  # Stop loss
        "tp": take_profit,  # Take profit
        "magic": 234000,  # Identificador mágico da ordem
        "type_time": mt5.ORDER_TIME_GTC,  # Tipo de expiração - GTC (Good Till Cancelled)
        "type_filling": mt5.ORDER_FILLING_RETURN,  # Política de preenchimento
    }

    # Enviar a ordem de compra
    result = mt5.order_send(order)
    
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Erro ao enviar a ordem de compra:", result.comment)
    else:
        print("Ordem de compra enviada com sucesso, id da ordem:", result.order)

def send_sell_limit_order(symbol, volume, price, stop_loss, take_profit):
    # Estrutura da Ordem de Venda
    order = {
        "action": mt5.TRADE_ACTION_DEAL,  # Ordem imediata
        "symbol": symbol,  # Símbolo do ativo
        "volume": volume,  # Volume da ordem
        "type": mt5.ORDER_TYPE_SELL,  # Tipo de ordem
        "price": price,  # Preço
        "sl": stop_loss,  # Stop loss
        "tp": take_profit,  # Take profit
        "magic": 234000,  # Identificador mágico da ordem
        "type_time": mt5.ORDER_TIME_GTC,  # Tipo de expiração - GTC (Good Till Cancelled)
        "type_filling": mt5.ORDER_FILLING_RETURN,  # Política de preenchimento
    }

    # Enviar a ordem de venda
    result = mt5.order_send(order)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Erro ao enviar a ordem de venda:", result.comment)
    else:
        print("Ordem de venda enviada com sucesso, id da ordem:", result.order)

### Função para Tomada de Decisão de Negócio no MetaTrader 5

Este código define a função `make_trade_decision`, que serve para tomar decisões automáticas de compra ou venda de um ativo, com base na previsão de seu preço de fechamento. A função realiza as seguintes etapas:

1. **Obtém o preço Bid e Ask do ativo**: Usa a função `mt5.symbol_info_tick()` para obter as informações de preço de compra (Bid) e de venda (Ask) do ativo.
   
2. **Calcula a previsão do preço de fechamento**: Realiza um cálculo de média ponderada entre o preço Ask e Bid, com uma pequena variação aleatória para simular a incerteza do mercado.

3. **Verifica condições de decisão**: Compara o preço previsto com os preços atuais (Bid e Ask) e decide se deve comprar ou vender:
   - Se o preço previsto for maior que o preço atual de compra e venda, a decisão é comprar.
   - Se o preço previsto for menor que ambos os preços, a decisão é vender.

4. **Calcula o Stop Loss e Take Profit**: Define os níveis de Stop Loss e Take Profit baseados no preço previsto de fechamento, ajustando de acordo com a direção da ordem (compra ou venda).

5. **Envia a ordem**: Dependendo da decisão (compra ou venda), a função chama as funções `send_buy_limit_order` ou `send_sell_limit_order` para executar a ordem na plataforma MetaTrader 5.

In [ ]:
# Função para Tomada de Decisão de Negócio no MetaTrader 5

def make_trade_decision(symbol, volume):

    # Obter os preços de Bid e Ask
    tick = mt5.symbol_info_tick(symbol)  # Pega os preços de Bid e Ask do símbolo
    if not tick:
        print(f"Erro ao obter informações para {symbol}")
        exit()
    else:
        # Armazenar os valores de Last Bid e Last Ask em variáveis
        current_bid = tick.bid
        current_ask = tick.ask

    # Calcula a previsão do preço de fechamento
    predicted_close = (current_ask * 0.7) + (current_bid * 0.3)  # Média ponderada
    predicted_close *= 1 + random.uniform(-0.001, 0.001)  # Pequena variação aleatória de ±0.1%

    # Verifica se o símbolo é 'USDJPY' para formatar com 3 casas decimais
    if symbol == "USDJPY":
        print(f"Previsão de fechamento calculada: {predicted_close:.3f}")
    else:
        print(f"Previsão de fechamento calculada: {predicted_close:.5f}")

    
    # Verifica se o preço previsto de fechamento é igual ao último preço de venda (ask) ou compra (bid)
    if predicted_close == current_ask or predicted_close == current_bid:
        print('Previsão é igual ao último valor de compra ou venda. Não faz nada.')
        return  # Se for igual, nenhuma ação é tomada e a função retorna.

    # Verifica se o preço previsto é igual tanto ao preço de compra quanto ao de venda
    if predicted_close == current_bid == current_ask:
        print('Preço previsto é igual ao preço de compra e venda. Não faz nada.')
        return  # Se o preço previsto for igual a ambos (bid e ask), nenhuma ação é tomada.

    # Se o preço previsto de fechamento for superior tanto ao preço de venda (ask) quanto ao de compra (bid)
    if predicted_close > current_ask and predicted_close > current_bid:
        order_type = "buy"  # Decisão é comprar
        price = current_ask  # O preço de compra será o preço de venda (ask)
    # Se o preço previsto de fechamento for superior ao preço de venda (ask) e igual ao preço de compra (bid)
    elif predicted_close > current_ask and predicted_close == current_bid:
        order_type = "buy"  # Decisão é comprar
        price = current_ask  # O preço de compra será o preço de venda (ask)
    # Se o preço previsto de fechamento for superior ao preço de compra (bid) e igual ao preço de venda (ask)
    elif predicted_close > current_bid and predicted_close == current_ask:
        order_type = "buy"  # Decisão é comprar
        price = current_ask  # O preço de compra será o preço de venda (ask)
    # Se o preço previsto de fechamento for inferior ao preço de venda (ask) e igual ao preço de compra (bid)
    elif predicted_close < current_ask and predicted_close == current_bid:
        order_type = "sell"  # Decisão é vender
        price = current_bid  # O preço de venda será o preço de compra (bid)
    # Se o preço previsto de fechamento for inferior tanto ao preço de venda (ask) quanto ao de compra (bid)
    elif predicted_close < current_ask and predicted_close < current_bid:
        order_type = "sell"  # Decisão é vender
        price = current_bid  # O preço de venda será o preço de compra (bid)
    # Se o preço previsto de fechamento for inferior ao preço de compra (bid) e superior ao preço de venda (ask)
    elif predicted_close < current_bid and predicted_close > current_ask:
        order_type = "sell"  # Decisão é vender
        price = current_bid  # O preço de venda será o preço de compra (bid)
    # Se o preço previsto de fechamento for superior tanto ao preço de compra quanto ao preço de venda, trata-se de uma decisão de compra
    elif predicted_close > current_bid and predicted_close > current_ask:
        order_type = "buy"  # Decisão é comprar
        price = current_ask  # O preço de compra será o preço de venda (ask)
    else:
        print("No valid trading decision could be made.")  # Caso nenhuma das condições anteriores seja atendida
        return  # Se não for possível tomar uma decisão, a função retorna.

    take_profit = predicted_close
    stop_loss = take_profit * 0.8 if order_type == "buy" else take_profit * 1.2 

    # Se a decisão for de compra
    if order_type == "buy":
        send_buy_limit_order(symbol, volume, price, stop_loss, take_profit)# Envia a ordem de compra
        print(f'Compra ao preço {price}. Venda (take profit) ao preço {take_profit}. Stop loss: {stop_loss}')
    else:
        send_sell_limit_order(symbol, volume, price, stop_loss, take_profit)  # Envia a ordem de venda
        print(f'Venda ao preço {price}. Compra (take profit) ao preço {take_profit}. Stop loss: {stop_loss}')

### Função Principal para Coletar Dados do MetaTrader 5

Este código define a função `main()`, que serve como ponto de entrada para a coleta de dados de um ativo no MetaTrader 5. Ele realiza as seguintes etapas:

1. **Conectar ao MetaTrader 5**: Chama a função `connect_mt5()` para estabelecer conexão com a plataforma.

2. **Entrada do usuário**:
   - Solicita ao usuário o símbolo do ativo (ex: EURUSD).
   - Permite a escolha de um timeframe (intervalo de tempo) entre opções como 1 minuto, 5 minutos, 1 hora, 1 dia, etc.
   - Pede o número de períodos desejados para a coleta de dados.

3. **Coleta de dados**: Utiliza a função `get_data()` para buscar os dados históricos do ativo com base nos parâmetros fornecidos.

4. **Exibe os dados coletados**: Imprime as informações obtidas no console.

5. **Opção de salvar os dados**: Chama a função `save_file(data)` para permitir que o usuário armazene os dados em um arquivo.

6. **Fecha a conexão com o MetaTrader 5**: Finaliza a conexão com a plataforma usando odelos preditivos.


In [ ]:
# Função principal
def main():
    # Conectar ao MetaTrader 5
    connect_mt5()

    # Entrada do usuário para o ativo
    symbol = input("Digite o símbolo do ativo (por exemplo, EURUSD): ")

    # Entrada do usuário para o timeframe
    print("Escolha o timeframe:")
    print("1 - M1 (1 minuto)")
    print("2 - M5 (5 minutos)")
    print("3 - M15 (15 minutos)")
    print("4 - M30 (30 minutos)")
    print("5 - H1 (1 hora)")
    print("6 - H4 (4 horas)")
    print("7 - D1 (1 dia)")
    print("8 - W1 (1 semana)")
    print("9 - MN1 (1 mês)")
    timeframe = int(input("Digite o número correspondente ao timeframe: "))

    # Entrada do usuário para o número de períodos
    periods = int(input("Digite o número de períodos desejados: "))

    # Coletar os dados
    data = get_data(symbol, timeframe, periods)
    
    # Exibir os dados
    print("\nDados coletados:")
    print(data)

    # Perguntar onde salvar
    save_file(data)

    # Fechar a conexão com o MetaTrader 5
    mt5.shutdown()

### Função para Coletar Dados Históricos do MetaTrader 5

A função `get_data()` coleta dados históricos de um ativo específico no MetaTrader 5 e os organiza em um DataFrame do Pandas. Ela realiza os seguintes passos:

1. **Mapeamento do Timeframe**: Associa os valores numéricos da escolha do usuário aos timeframes do MetaTrader 5.

2. **Coleta de Dados**: Utiliza `mt5.copy_rates_from_pos()` para obter os dados históricos do ativo, a partir da posição mais recente.

3. **Criação do DataFrame**: Converte os dados obtidos em um `DataFrame` do Pandas, incluindo a formatação da coluna `time` para um formato de data legível (`DATE`).

4. **Organização das Colunas**: Remove a coluna original `time` e define a ordem das colunas (`DATE`, `open`, `high`, `low`, `close`, `tick_volume`, `spread`). Se a coluna `volume` estiver presente, ela é incluída.

5. **Retorno dos Dados**: Retorna o `DataFrame` organizado para ser utilizado em análises ou estatística.


In [ ]:
# Função para coletar dados históricos
def get_data(symbol, timeframe, periods):
    # Define o timeframe
    timeframes = {
        1: mt5.TIMEFRAME_M1,
        2: mt5.TIMEFRAME_M5,
        3: mt5.TIMEFRAME_M15,
        4: mt5.TIMEFRAME_M30,
        5: mt5.TIMEFRAME_H1,
        6: mt5.TIMEFRAME_H4,
        7: mt5.TIMEFRAME_D1,
        8: mt5.TIMEFRAME_W1,
        9: mt5.TIMEFRAME_MN1
    }

    # Coleta os dados históricos
    rates = mt5.copy_rates_from_pos(symbol, timeframes[timeframe], 0, periods)
    
    # Cria o dataframe
    data = pd.DataFrame(rates)
    data['DATE'] = pd.to_datetime(data['time'], unit='s')
    data.drop(columns=['time'], inplace=True)  # Remove a coluna 'time'
    
    # Reorganiza as colunas, considerando a possível ausência de 'volume'
    columns = ['DATE', 'open', 'high', 'low', 'close', 'tick_volume', 'spread']
    
    # Verifica se a coluna 'volume' está presente, se não, a remove
    if 'volume' in data.columns:
        columns.append('volume')
    
    data = data[columns]
    
    return data

### Função para Salvar os Dados em Arquivos `.csv` ou `.xlsx`

A função `save_file(data)` permite ao usuário salvar os dados coletados em um arquivo `.csv` ou `.xlsx`. Ela funciona da seguinte forma:

1. **Interface Gráfica**: Utiliza `tkinter` para abrir um diálogo de seleção de arquivo, permitindo que o usuário escolha onde salvar os dados.

2. **Extensão Padrão e Tipos de Arquivo**: O arquivo será salvo como `.csv` por padrão, mas o usuário pode optar por `.xlsx` (Excel).

3. **Salvamento dos Dados**:
   - Se o usuário escolher `.csv`, os dados são salvos usando `to_csv()`.
   - Se escolher `.xlsx`, são salvos usando `to_excel()` com o engine `openpyxl`.

4. **Confirmação**: Exibe no console o caminho onde o arquivras análises.


In [ ]:
# Função para salvar os dados em .csv ou .excel
def save_file(data):
    # Configura a interface gráfica de seleção de arquivo
    root = tk.Tk()
    root.withdraw()  # Não exibe a janela principal

    # Pergunta onde o usuário deseja salvar o arquivo
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")])
    
    if file_path:
        if file_path.endswith('.csv'):
            data.to_csv(file_path, index=False)
        elif file_path.endswith('.xlsx'):
            data.to_excel(file_path, index=False, engine='openpyxl')
        print(f"Arquivo salvo em: {file_path}")

Este código define uma função `get_loop_interval` que retorna um intervalo de tempo em segundos com base no `timeframe` fornecido. O `timeframe` representa diferentes períodos de tempo, como minutos (M1, M5, etc.), horas (H1, H4), dias (D1), semanas (W1) e meses (MN1). A função utiliza um dicionário para mapear cada `timeframe` ao seu respectivo intervalo de tempo em segundos. Se o `timeframe` fornecido não estiver no dicionário, a função retorna um valor padrão de 60 segundos (1 minuto).

In [ ]:
def get_loop_interval(timeframe):
    """Retorna o intervalo de tempo do loop com base no timeframe selecionado."""
    intervals = {
        1: 15,       # M1 -> 15 segundos
        5: 60,       # M5 -> 1 minuto
        15: 300,     # M15 -> 5 minutos
        30: 900,     # M30 -> 15 minutos
        60: 1800,    # H1 -> 30 minutos
        240: 3600,   # H4 -> 1 hora
        1440: 14400, # D1 -> 4 horas
        10080: 86400,# W1 -> 24 horas
        43200: 604800# MN1 -> 1 semana
    }
    return intervals.get(timeframe, 60)  # Padrão: 1 minuto caso o timeframe seja inválido

Este código define uma função `clear_console` que limpa a saída do console em um ambiente de Jupyter Notebook. A função utiliza `clear_output(wait=True)` da biblioteca `IPython.display`, que remove o conteúdo exibido anteriormente na célula de saída. O parâmetro `wait=True` garante que a limpeza ocorra apenas após a execução de novos comandos, evitando comportamentos indesejados, como piscar a tela.

In [ ]:
def clear_console():
    """Limpa a saída do console no Jupyter Notebook."""
    clear_output(wait=True)

## Executar Ordens de Negociação

Este código define uma função `executar_negociacao` que implementa um sistema de negociação automatizada no MetaTrader 5 (MT5). A função realiza as seguintes etapas:

1. **Inicialização do MT5**: Verifica se o MetaTrader 5 foi inicializado corretamente. Caso contrário, exibe uma mensagem de erro e encerra.

2. **Entrada do usuário**: Solicita ao usuário o símbolo do ativo (por exemplo, EURUSD) e o timeframe desejado (como M1, H1, D1, etc.). O timeframe é mapeado para um valor numérico correspondente.

3. **Validação do volume**: Solicita o volume da negociação e valida se está dentro do intervalo permitido (0.00 a 500.00).

4. **Loop de negociação**: Entra em um loop infinito onde:
   - Verifica se o mercado está aberto para o ativo selecionado.
   - Verifica se há ordens abertas para o ativo.
   - Se o mercado estiver aberto e não houver ordens abertas, executa uma decisão de negociação (`make_trade_decision`).
   - Aguarda um intervalo de tempo (`loop_interval`) antes de repetir o processo.

5. **Intervalo de loop**: O intervalo de tempo é definido com base no timeframe escolhido, utilizando a função `get_loop_interval`.

6. **Limpeza do console**: A cada iteração, o console é limpo para manter a saída organizada.

A função é projetada para operar continuamente, tomando decisões de negociação com base nas condições do mercado e nas entradas do usuário.

In [ ]:
def executar_negociacao():
    """Executa a negociação automatizada com base no timeframe definido pelo usuário."""
    if not mt5.initialize():
        print("Erro ao inicializar MetaTrader 5.")
        mt5.shutdown()
        return

    symbol = input("Digite o símbolo do ativo (por exemplo, EURUSD): ")
    
    print("Escolha o timeframe:")
    print("1 - M1 (1 minuto)")
    print("2 - M5 (5 minutos)")
    print("3 - M15 (15 minutos)")
    print("4 - M30 (30 minutos)")
    print("5 - H1 (1 hora)")
    print("6 - H4 (4 horas)")
    print("7 - D1 (1 dia)")
    print("8 - W1 (1 semana)")
    print("9 - MN1 (1 mês)")
    
    timeframe_options = {1: 1, 2: 5, 3: 15, 4: 30, 5: 60, 6: 240, 7: 1440, 8: 10080, 9: 43200}
    
    try:
        timeframe_choice = int(input("Digite o número correspondente ao timeframe: "))
        timeframe = timeframe_options.get(timeframe_choice, 60)  # Padrão: H1 (1 hora) caso inválido
    except ValueError:
        print("Erro: entrada inválida para o timeframe. Usando H1 como padrão.")
        timeframe = 60

    loop_interval = get_loop_interval(timeframe)
    if loop_interval is None or loop_interval <= 0:
        print("Erro ao obter o intervalo de loop. Usando 60 segundos como padrão.")
        loop_interval = 60

    volume_input = input("Digite o volume desejado (entre 0.00 e 500.00): ")
    volume = validar_volume(volume_input)
    if volume is None:
        print("Volume inválido. Saindo...")
        return

    while True:
        clear_console()

        mercado_aberto = is_market_open(symbol)
        ordens_abertas = check_open_positions(symbol)

        # Debug para verificar valores antes de tomar decisões
        print(f"DEBUG: Mercado aberto? {mercado_aberto} | Ordens abertas? {ordens_abertas}")

        if not mercado_aberto:
            print(f"O mercado para {symbol} está fechado. Aguardando...")
        elif ordens_abertas:
            print(f"Já existem ordens abertas para {symbol}. Aguardando...")
        else:
            print(f"Iniciando negociação para {symbol}...")
            make_trade_decision(symbol, volume)
            print(f"Negociação executada. Aguardando {loop_interval} segundos para a próxima verificação...")

        time.sleep(loop_interval)

### Função para Verificar se o Mercado do Ativo Está Aberto

A função `is_market_open(symbol: str) -> bool` verifica se um determinado ativo está disponível para negociação no MetaTrader 5. O processo ocorre em três etapas:

1. **Verificação da Existência do Ativo**: 
   - A função obtém informações do ativo através de `mt5.symbol_info(symbol)`.
   - Se o ativo não for encontrado, retorna `False` e exibe uma mensagem de erro.

2. **Verificação da Disponibilidade para Trading**:
   - Se o ativo não estiver visível (`symbol_info.visible == False`), significa que ele não está habilitado para negociação no MetaTrader 5.

3. **Verificação do Horário do Mercado**:
   - A função verifica se é um dia útil (segunda a sexta-feira). Se for sábado ou domingo, retorna `False`.
   - Se todas as condições forem atendidas, retorna `True`, indicando que o ativo está disponível par MetaTrader 5.


In [ ]:
# Função para verificar se o mercado do ativo está aberto
def is_market_open(symbol: str) -> bool:
    symbol_info = mt5.symbol_info(symbol)
    
    if symbol_info is None:
        print(f"Ativo {symbol} não encontrado no MetaTrader 5.")
        return False

    # Verificar se o ativo está habilitado para trading
    if not symbol_info.visible:
        print(f"Ativo {symbol} não está habilitado para trading no MetaTrader 5.")
        return False

    # Verificar se é um dia útil (segunda a sexta-feira)
    now = datetime.now()
    if now.weekday() >= 5:  # Sábado (5) ou domingo (6)
        return False

    # Verificar se o mercado está disponível para negociação
    return True

### Função para Verificar Ordens Abertas para um Ativo

A função `check_open_positions(symbol: str)` verifica se há ordens abertas para um determinado ativo no MetaTrader 5. O processo ocorre da seguinte maneira:

1. **Obtenção das Posições Abertas**:
   - Utiliza `mt5.positions_get()` para recuperar todas as posições abertas no momento.

2. **Tratamento de Erros e Verificação**:
   - Se `positions` for `None`, exibe uma mensagem indicando erro na obtenção dos dados.
   - Se `positions` estiver vazio, informa que não há ordens abertas.

3. **Busca por Ordens do Ativo Específico**:
   - Percorre todas as posições abertas e verifica se o ativo corresponde ao símbolo informado.
   - Se encontrar, exibe detalhes como:
     - **Símbolo** do ativo.
     - **Volume** da posição.
     - **Tipo da Ordem** (compra ou venda).

4. **Resultado Final**:
   - Se houver ordens abertas para o ativo, informa isso ao usuário.
   - Caso contrário, exibe uma mensagem informando que não há posições abertas para para um mesmo ativo.

In [ ]:
# Função para verificar ordens abertas para o ativo escolhido
def check_open_positions(symbol: str):
    # Obter todas as posições abertas
    positions = mt5.positions_get()

    if positions is None:
        print("Erro ao obter as posições abertas.")
    elif len(positions) == 0:
        print("Não há ordens abertas no momento.")
    else:
        # Flag para verificar se o ativo escolhido tem ordens abertas
        ativo_em_aberto = False
        print(f"Verificando ordens abertas para o ativo {symbol}:")
        for position in positions:
            if position.symbol == symbol:
                ativo_em_aberto = True
                # Exibir detalhes da posição
                print(f"Ativo: {position.symbol}, Volume: {position.volume}, Tipo: {'Compra' if position.type == 0 else 'Venda'}")
        
        if not ativo_em_aberto:
            print(f"Não há ordens abertas para o ativo {symbol}.")
        else:
            print(f"Existem ordens abertas para o ativo {symbol}.")

### Função para Validar e Ajustar o Volume

A função `validar_volume(volume_input)` garante que o volume inserido pelo usuário esteja no formato correto e dentro dos limites permitidos. O funcionamento é o seguinte:

1. **Conversão para Float**: A entrada do usuário é convertida em um número decimal (`float`).
2. **Precisão Decimal**: O volume é arredondado para duas casas decimais.
3. **Validação de Intervalo**:
   - O valor deve estar entre `0.01` e `500.00` (excluindo `0.00`).
   - Se estiver fora desse intervalo, o usuário recebe um aviso e deve inserir um novo valor.
4. **Tratamento de Erros**: Se o usuário inserir um valor inválido (como texto ou caracteres não numéricos), o programa exibe uma mensagem de erro e solicita um noaTrader 5.


In [ ]:
# Função para garantir que o volume esteja no formato correto com dois dígitos após o ponto
def validar_volume(volume_input):
    while True:
        try:
            volume = float(volume_input)  # Converte a entrada para float
            # Limita a 2 casas decimais
            volume = round(volume, 2)
            # Verifica se o volume está dentro do intervalo desejado
            if 0.00 < volume <= 500.00:
                return volume
            else:
                print("Volume fora do intervalo permitido (0.00 a 500.00).")
                volume_input = input("Digite o volume desejado (entre 0.00 e 500.00): ")  # Solicita novo valor
        except ValueError:
            print("Valor de volume inválido. Certifique-se de inserir um número válido.")
            volume_input = input("Digite o volume desejado (entre 0.00 e 500.00): ")  # Solicita novo valor